# The Daubechies D4 Wavelet Tranform

In [85]:
import numpy as np

root2, root3 = np.sqrt(2), np.sqrt(3) 

# h0 = (1 + root3) / (4 * root2)
# h1 = (3 + root3) / (4 * root2)
# h2 = (3 - root3) / (4 * root2)
# h3 = (1 - root3) / (4 * root2)
# h_coeffs = np.array([h0,h1,h2,h3])

# g0 = h3
# g1 = -h2
# g2 = h1
# g3 = -h0
# g_coeffs = np.array([g0,g1,g2,g3])

h_coeffs = np.array([(1 + root3) / (4 * root2),
                     (3 + root3) / (4 * root2),
                     (3 - root3) / (4 * root2),
                     (1 - root3) / (4 * root2)])

g_coeffs = np.array([h_coeffs[3], -h_coeffs[2], h_coeffs[1], -h_coeffs[0]])

daubechies_4 = np.zeros((8, 8))

In [86]:
def daubechies4_wavelet_transform_2D(image): # ESTE FUNCIONA
    rows, cols = image.shape
    matrix_coeffs = np.zeros((rows, cols))

    for i in range(rows // 2):
        if i < rows // 2 - 1:
            matrix_coeffs[i, 2 * i:2 * i + 4] = h_coeffs
        else:
            matrix_coeffs[i, 2 * i:2 * i + 2] = h_coeffs[:2]
            matrix_coeffs[i, 0:2] = h_coeffs[2:]

    for i in range(rows // 2):
        if i < rows // 2 - 1:
            matrix_coeffs[i + rows // 2, 2 * i:2 * i + 4] = g_coeffs
        else:
            matrix_coeffs[i + rows // 2, 2 * i:2 * i + 2] = g_coeffs[:2]
            matrix_coeffs[i + rows // 2, 0:2] = g_coeffs[2:]

    return(np.round(matrix_coeffs, 3))

In [87]:
daubechies_4 = daubechies4_wavelet_transform_2D(daubechies_4)
print(daubechies_4)
column = np.array([32, 32, 16, 8, 24, 16, 64, 32])

[[ 0.483  0.837  0.224 -0.129  0.     0.     0.     0.   ]
 [ 0.     0.     0.483  0.837  0.224 -0.129  0.     0.   ]
 [ 0.     0.     0.     0.     0.483  0.837  0.224 -0.129]
 [ 0.224 -0.129  0.     0.     0.     0.     0.483  0.837]
 [-0.129 -0.224  0.837 -0.483  0.     0.     0.     0.   ]
 [ 0.     0.    -0.129 -0.224  0.837 -0.483  0.     0.   ]
 [ 0.     0.     0.     0.    -0.129 -0.224  0.837 -0.483]
 [ 0.837 -0.483  0.     0.     0.     0.    -0.129 -0.224]]


In [91]:
# Obtener el número de filas de la matriz
num_filas = daubechies_4.shape[0]

# Crear una daubechies_4 vacía para almacenar el resultado
resultado = np.zeros_like(daubechies_4)

# Realizar la multiplicación de cada fila de la daubechies_4 por el valor correspondiente del vector
for fila in range(num_filas):
    resultado[fila, :] = daubechies_4[fila, :] * column[fila]


print(resultado)

[[ 15.456  26.784   7.168  -4.128   0.      0.      0.      0.   ]
 [  0.      0.     15.456  26.784   7.168  -4.128   0.      0.   ]
 [  0.      0.      0.      0.      7.728  13.392   3.584  -2.064]
 [  1.792  -1.032   0.      0.      0.      0.      3.864   6.696]
 [ -3.096  -5.376  20.088 -11.592   0.      0.      0.      0.   ]
 [  0.      0.     -2.064  -3.584  13.392  -7.728   0.      0.   ]
 [  0.      0.      0.      0.     -8.256 -14.336  53.568 -30.912]
 [ 26.784 -15.456   0.      0.      0.      0.     -4.128  -7.168]]


## IMAGES EXAMPLES

In [89]:
import cv2
import IPython
from PIL import Image
import matplotlib.pyplot as plt

def imshow(image, width=None):
    _, ret = cv2.imencode('.jpg', image)
    i = IPython.display.Image(data=ret, width=width)
    IPython.display.display(i)

def jpg_to_pgm(jpg_path):
    image = Image.open(jpg_path)
    image_gray = image.convert('L')
    width = image_gray.size[0]
    if (width>512):
        image_gray = image_gray.resize((512, 512))
        pixels = list(image_gray.getdata())

    else: pixels = list(image_gray.getdata())

    return np.array(pixels).reshape((512, 512))

def p_new(matrix):
    max,min  = np.max(matrix), np.min(matrix)
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            matrix[i][j] = ((matrix[i][j]-min)*255.0)/(max-min)
    return matrix